# Weather Feature Engineering

Enriching the dataset with weather and environmental features:
- Historical rainfall and temperature data
- Evapotranspiration estimates
- Weather anomalies (deviation from long-term means)
- Daylength and seasonal features
- NDVI anomalies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our custom weather API
from weather_api import (
    WeatherAPIClient, 
    calculate_daylength, 
    get_southern_hemisphere_season
)

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

## 1. Load Training Data

In [ ]:
# Load training data
train_df = pd.read_csv('competition/train.csv')

# Convert to wide format
train_wide = train_df.pivot_table(
    index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
    columns='target_name',
    values='target'
).reset_index()

# Convert date
train_wide['Sampling_Date'] = pd.to_datetime(train_wide['Sampling_Date'])

print(f"Training samples: {len(train_wide)}")
print(f"\nStates: {train_wide['State'].value_counts().to_dict()}")
print(f"\nDate range: {train_wide['Sampling_Date'].min()} to {train_wide['Sampling_Date'].max()}")
train_wide.head()

## 2. Initialize Weather API Client

In [ ]:
# Initialize weather client with caching
weather_client = WeatherAPIClient(cache_dir='weather_cache')

print("Weather API Client initialized")
print("\nRepresentative locations:")
for state, info in weather_client.LOCATIONS.items():
    print(f"  {state}: {info['name']} ({info['lat']:.2f}°, {info['lon']:.2f}°)")

## 3. Fetch Weather Data for All States

This will fetch historical weather data for the date range in our dataset.

In [ ]:
# Get overall date range (with buffer for rolling calculations)
min_date = train_wide['Sampling_Date'].min()
max_date = train_wide['Sampling_Date'].max()

print(f"Fetching weather data from {min_date.date()} to {max_date.date()}")
print("(includes 30 days before for rolling averages)\n")

# Fetch data for each state
weather_data = {}
for state in train_wide['State'].unique():
    print(f"\n{state}:")
    df = weather_client.fetch_weather_data(
        state=state,
        start_date=min_date.strftime('%Y-%m-%d'),
        end_date=max_date.strftime('%Y-%m-%d'),
        days_before=30
    )
    weather_data[state] = df
    print(f"  → {len(df)} days fetched")

print("\n✓ All weather data fetched and cached!")

In [ ]:
# Preview weather data
print("Sample weather data for Tasmania:")
weather_data['Tas'].head(10)

## 4. Calculate Rolling Weather Features

Create 7-day and 30-day rolling averages.

In [ ]:
def calculate_rolling_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate rolling weather features.
    
    Args:
        df: Weather dataframe with daily data
    
    Returns:
        DataFrame with rolling features added
    """
    df = df.copy().sort_values('date')
    
    # 7-day rolling features
    df['rainfall_7d'] = df['precipitation'].rolling(window=7, min_periods=1).sum()
    df['temp_max_7d'] = df['temp_max'].rolling(window=7, min_periods=1).mean()
    df['temp_min_7d'] = df['temp_min'].rolling(window=7, min_periods=1).mean()
    df['temp_mean_7d'] = df['temp_mean'].rolling(window=7, min_periods=1).mean()
    df['et0_7d'] = df['et0'].rolling(window=7, min_periods=1).sum()
    
    # 30-day rolling features
    df['rainfall_30d'] = df['precipitation'].rolling(window=30, min_periods=1).sum()
    df['temp_mean_30d'] = df['temp_mean'].rolling(window=30, min_periods=1).mean()
    df['et0_30d'] = df['et0'].rolling(window=30, min_periods=1).sum()
    
    # Temperature range (daily and 7-day avg)
    df['temp_range'] = df['temp_max'] - df['temp_min']
    df['temp_range_7d'] = df['temp_range'].rolling(window=7, min_periods=1).mean()
    
    # Days since significant rain (>5mm)
    df['significant_rain'] = (df['precipitation'] > 5).astype(int)
    df['days_since_rain'] = 0
    
    days_counter = 0
    days_since_rain = []
    for has_rain in df['significant_rain']:
        if has_rain:
            days_counter = 0
        else:
            days_counter += 1
        days_since_rain.append(days_counter)
    
    df['days_since_rain'] = days_since_rain
    
    # Water balance (rainfall - ET)
    df['water_balance_7d'] = df['rainfall_7d'] - df['et0_7d']
    df['water_balance_30d'] = df['rainfall_30d'] - df['et0_30d']
    
    return df

# Calculate rolling features for each state
print("Calculating rolling features...\n")
for state in weather_data.keys():
    weather_data[state] = calculate_rolling_features(weather_data[state])
    print(f"✓ {state}: {len(weather_data[state])} days processed")

print("\nRolling features calculated!")

In [ ]:
# Preview rolling features
print("Rolling features for Tasmania (last 10 days):")
cols_to_show = ['date', 'precipitation', 'rainfall_7d', 'rainfall_30d', 
                'temp_mean', 'temp_mean_7d', 'et0_7d', 'days_since_rain']
weather_data['Tas'][cols_to_show].tail(10)

# Combine all weather data
all_weather = pd.concat(weather_data.values(), ignore_index=True)

# Check what columns we actually have
print("Available columns in weather data:")
print(all_weather.columns.tolist())
print(f"\nTotal columns: {len(all_weather.columns)}")

# Select features that exist in the data
available_features = [
    'date', 'state',
    # Rolling rainfall
    'rainfall_7d', 'rainfall_30d',
    # Rolling temperature
    'temp_max_7d', 'temp_min_7d', 'temp_range_7d',
    # Evapotranspiration
    'et0_7d', 'et0_30d',
    # Water balance
    'water_balance_7d', 'water_balance_30d',
    # Days since rain
    'days_since_rain',
    # Astronomical (if available)
    'daylength', 'season'
]

# Only include features that exist
weather_features = ['date', 'state']
for feat in available_features[2:]:
    if feat in all_weather.columns:
        weather_features.append(feat)
    else:
        print(f"⚠ Feature '{feat}' not found, skipping")

# Add anomalies if they exist
if 'rain_anomaly' in all_weather.columns:
    weather_features.extend(['rain_anomaly', 'temp_anomaly', 'et0_anomaly'])
else:
    print("⚠ Anomaly features not calculated (climatology missing), skipping")

weather_for_merge = all_weather[weather_features].copy()
weather_for_merge.columns = ['Sampling_Date', 'State'] + weather_features[2:]

print(f"\nWeather features prepared: {len(weather_features) - 2} features")
print(f"Features: {weather_features[2:]}")

In [ ]:
# Fetch long-term climatology (1991-2020)
print("Fetching climatology data for anomaly calculations...\n")

climatology_data = {}
for state in weather_data.keys():
    print(f"{state}:")
    clima = weather_client.fetch_climatology(state, start_year=1991, end_year=2020)
    climatology_data[state] = clima
    print(f"  ✓ Climatology loaded\n")

print("Climatology data ready!")

In [ ]:
# Preview climatology
print("Climatology for Tasmania (sample):")
climatology_data['Tas'].head()

In [ ]:
# List all new features that were actually added
new_features = [col for col in train_enriched.columns if col not in train_wide.columns]

print(f"Total new features: {len(new_features)}")
print(f"\nNew features added:")
for feat in new_features:
    print(f"  - {feat}")

print("\n" + "="*60)
print("Feature summary:")
print("="*60)

# Get summary stats for numeric features only
numeric_new_features = [f for f in new_features if train_enriched[f].dtype in ['float64', 'int64']]
if numeric_new_features:
    print(train_enriched[numeric_new_features].describe().T)

## 6. Calculate Daylength and Season Features

In [ ]:
def add_astronomical_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add daylength and season features.
    
    Args:
        df: Weather dataframe with 'date' and 'lat' columns
    
    Returns:
        DataFrame with daylength and season added
    """
    df = df.copy()
    
    # Calculate daylength for each row
    df['daylength'] = df.apply(
        lambda row: calculate_daylength(row['lat'], row['date']),
        axis=1
    )
    
    # Add season (Southern Hemisphere)
    df['season'] = df['date'].apply(get_southern_hemisphere_season)
    
    return df

# Add astronomical features
print("Calculating daylength and season...\n")
for state in weather_data.keys():
    weather_data[state] = add_astronomical_features(weather_data[state])
    print(f"✓ {state}: Astronomical features added")

print("\nDaylength and season calculated!")

## 7. Merge Weather Features with Training Data

In [ ]:
# Combine all weather data
all_weather = pd.concat(weather_data.values(), ignore_index=True)

# Select features to merge
weather_features = [
    'date', 'state',
    # Rolling rainfall
    'rainfall_7d', 'rainfall_30d',
    # Rolling temperature
    'temp_max_7d', 'temp_min_7d', 'temp_mean_7d', 'temp_mean_30d', 'temp_range_7d',
    # Evapotranspiration
    'et0_7d', 'et0_30d',
    # Water balance
    'water_balance_7d', 'water_balance_30d',
    # Days since rain
    'days_since_rain',
    # Anomalies
    'rain_anomaly', 'temp_anomaly', 'et0_anomaly',
    # Astronomical
    'daylength', 'season'
]

weather_for_merge = all_weather[weather_features].copy()
weather_for_merge.columns = ['Sampling_Date', 'State'] + weather_features[2:]

print(f"Weather features prepared: {len(weather_features) - 2} features")
print(f"Features: {weather_features[2:]}")

In [ ]:
# Merge with training data
train_enriched = train_wide.merge(
    weather_for_merge,
    on=['Sampling_Date', 'State'],
    how='left'
)

print(f"Original columns: {train_wide.shape[1]}")
print(f"Enriched columns: {train_enriched.shape[1]}")
print(f"New features added: {train_enriched.shape[1] - train_wide.shape[1]}")
print(f"\nRows with missing weather data: {train_enriched[weather_features[2:]].isnull().any(axis=1).sum()}")

train_enriched.head()

## 8. Calculate NDVI Anomaly

Compare each NDVI reading to the rolling mean for that state.

In [ ]:
# Calculate NDVI statistics by state
ndvi_stats = train_enriched.groupby('State')['Pre_GSHH_NDVI'].agg(['mean', 'std']).reset_index()
ndvi_stats.columns = ['State', 'ndvi_mean_state', 'ndvi_std_state']

# Merge and calculate anomaly
train_enriched = train_enriched.merge(ndvi_stats, on='State', how='left')
train_enriched['ndvi_anomaly'] = (
    (train_enriched['Pre_GSHH_NDVI'] - train_enriched['ndvi_mean_state']) / 
    train_enriched['ndvi_std_state']
)

print("NDVI anomaly calculated")
print("\nNDVI statistics by state:")
print(ndvi_stats)

## 9. Feature Summary and Visualization

In [ ]:
# List all new features
new_features = [
    'rainfall_7d', 'rainfall_30d',
    'temp_max_7d', 'temp_min_7d', 'temp_mean_7d', 'temp_mean_30d', 'temp_range_7d',
    'et0_7d', 'et0_30d',
    'water_balance_7d', 'water_balance_30d',
    'days_since_rain',
    'rain_anomaly', 'temp_anomaly', 'et0_anomaly',
    'daylength', 'season', 'ndvi_anomaly'
]

print(f"Total new features: {len(new_features)}")
print("\nFeature summary:")
print(train_enriched[new_features].describe())

In [ ]:
# Visualize correlations with target variables
target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

# Select key weather features for correlation
key_weather_features = [
    'rainfall_7d', 'rainfall_30d', 'temp_mean_7d', 'et0_7d',
    'water_balance_7d', 'days_since_rain', 'rain_anomaly',
    'temp_anomaly', 'daylength', 'ndvi_anomaly'
]

# Calculate correlations
corr_data = train_enriched[key_weather_features + target_cols].corr()
target_corr = corr_data.loc[key_weather_features, target_cols]

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(target_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            cbar_kws={'label': 'Correlation'})
plt.title('Weather Feature Correlations with Biomass Targets')
plt.tight_layout()
plt.show()

print("\nTop 5 features correlated with Dry_Total_g:")
total_corr = target_corr['Dry_Total_g'].abs().sort_values(ascending=False)
print(total_corr.head())

In [ ]:
# Visualize weather patterns by state
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Rainfall
axes[0, 0].boxplot([train_enriched[train_enriched['State']==s]['rainfall_30d'].values 
                     for s in ['Tas', 'Vic', 'NSW', 'WA']],
                    labels=['Tas', 'Vic', 'NSW', 'WA'])
axes[0, 0].set_ylabel('30-day Rainfall (mm)')
axes[0, 0].set_title('Rainfall by State')
axes[0, 0].grid(True, alpha=0.3)

# Temperature
axes[0, 1].boxplot([train_enriched[train_enriched['State']==s]['temp_mean_7d'].values 
                     for s in ['Tas', 'Vic', 'NSW', 'WA']],
                    labels=['Tas', 'Vic', 'NSW', 'WA'])
axes[0, 1].set_ylabel('7-day Mean Temp (°C)')
axes[0, 1].set_title('Temperature by State')
axes[0, 1].grid(True, alpha=0.3)

# Daylength
for state in ['Tas', 'Vic', 'NSW', 'WA']:
    state_data = train_enriched[train_enriched['State']==state].sort_values('Sampling_Date')
    axes[1, 0].plot(state_data['Sampling_Date'], state_data['daylength'], 
                    marker='o', label=state, alpha=0.7)
axes[1, 0].set_ylabel('Daylength (hours)')
axes[1, 0].set_title('Daylength Over Time')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Water balance
axes[1, 1].boxplot([train_enriched[train_enriched['State']==s]['water_balance_7d'].values 
                     for s in ['Tas', 'Vic', 'NSW', 'WA']],
                    labels=['Tas', 'Vic', 'NSW', 'WA'])
axes[1, 1].set_ylabel('7-day Water Balance (mm)')
axes[1, 1].set_title('Water Balance by State')
axes[1, 1].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Save Enriched Dataset

In [ ]:
# Save enriched training data
output_file = 'competition/train_enriched.csv'
train_enriched.to_csv(output_file, index=False)

print(f"✓ Enriched dataset saved to {output_file}")
print(f"\nDataset shape: {train_enriched.shape}")
print(f"Total features: {train_enriched.shape[1]}")
print(f"Weather features added: {len(new_features)}")

# Save feature list for reference
feature_info = pd.DataFrame({
    'feature': new_features,
    'description': [
        '7-day total rainfall (mm)',
        '30-day total rainfall (mm)',
        '7-day avg max temperature (°C)',
        '7-day avg min temperature (°C)',
        '7-day avg mean temperature (°C)',
        '30-day avg mean temperature (°C)',
        '7-day avg daily temperature range (°C)',
        '7-day total evapotranspiration (mm)',
        '30-day total evapotranspiration (mm)',
        '7-day water balance: rain - ET (mm)',
        '30-day water balance: rain - ET (mm)',
        'Days since last >5mm rainfall',
        'Rainfall anomaly (% from long-term mean)',
        'Temperature anomaly (°C from long-term mean)',
        'ET anomaly (% from long-term mean)',
        'Hours of daylight',
        'Season (0=Summer, 1=Autumn, 2=Winter, 3=Spring)',
        'NDVI standardized anomaly (z-score)'
    ]
})

feature_info.to_csv('weather_features_info.csv', index=False)
print("\n✓ Feature documentation saved to weather_features_info.csv")

## Summary

### Weather Features Added:

**Rainfall (3 features)**:
- 7-day and 30-day totals
- Rainfall anomaly (% from climatology)

**Temperature (5 features)**:
- 7-day avg max, min, mean
- 30-day avg mean
- 7-day avg daily range
- Temperature anomaly

**Evapotranspiration (3 features)**:
- 7-day and 30-day totals
- ET anomaly

**Water Balance (2 features)**:
- 7-day and 30-day (rainfall - ET)

**Drought Indicator (1 feature)**:
- Days since last significant rain

**Astronomical (2 features)**:
- Daylength (hours)
- Season (0-3)

**NDVI (1 feature)**:
- NDVI anomaly (z-score)

**Total: 18 new features**

### Expected Impact:
Weather features should significantly improve model performance, especially for:
- **Dry_Total_g**: Highly correlated with recent rainfall and water balance
- **Dry_Green_g**: Related to growing conditions (temp, water, daylength)
- **GDM_g**: Active growth depends on favorable weather

### Next Steps:
1. Update multimodal model to use these new features
2. Retrain and evaluate performance improvement
3. Feature importance analysis
4. Handle test data (will need to fetch weather for test dates)